In [ ]:
import re
import nltk
import time
import pandas as pd
import numpy as np
import sklearn
from nltk import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from pattern.en import singularize
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
stop = stopwords.words('english')

In [ ]:
#Import Data
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
attributes=pd.read_csv('attributes.csv')
description=pd.read_csv('product_descriptions.csv')

In [ ]:
description.index=description['product_uid']
description=description.drop('product_uid',axis=1)
#Remove Nan
attributes=attributes.dropna(how='all')
attributes.reset_index(inplace=True)

In [ ]:
#spell correcting

from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter('\n'.join(s for s in description.as_matrix().flatten()).split())

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
stop=stopwords.words('english')
def cleaning_text(sentence):
    if isinstance(sentence,basestring):
        sentence = unicode(sentence,'utf-8', errors='replace')
        sentence=sentence.lower()
        sentence=re.sub('[^\w\s]',' ', sentence) #removes punctuations
        sentence=re.sub('_', ' ', sentence) #removes punctuations
        sentence=re.sub('\d+',' ', sentence) #removes digits
        cleaned=' '.join([correction(w) for w in sentence.split() if not w in stop]) 
        cleaned=' '.join([singularize(w) for w in sentence.split() ]) #remove pural
        #removes english stopwords
        cleaned=' '.join([w for w , pos in pos_tag(cleaned.split()) if (pos == 'NN' or pos=='JJ' or pos=='JJR' or pos=='JJS' )])
        #selecting only nouns and adjectives

        cleaned=' '.join([w for w in cleaned.split() if not len(w)<=2 ]) 
        #removes single lettered words and digits
        cleaned=cleaned.strip()
        return cleaned
    else:
        return ''

In [ ]:
#Bullets_cleaned=Bullets['Bullets'].apply(lambda x: cleaning_text(x))
Description_cleaned=description['product_description'].apply(lambda x: cleaning_text(x))

pickle.dump(Description_cleaned, open( "Description_cleaned.p", "wb" ))
#Description_cleaned=pickle.load(open( "Description_cleaned.p", "rb" ))

In [ ]:
Description_cleaned.index=description.index

In [ ]:
#Search_Term=pd.Series(train.search_term.append(test.search_term))
Search_Term=Search_Term.apply(lambda x: cleaning_text(x))
pickle.dump(Search_Term,open('Search_Term.p','wb'))
#Search_Term=pickle.load(open('Search_Term.p','rb'))

In [8]:
tf_vectorizer1 = CountVectorizer(max_df=0.95, min_df=2, max_features=200,
                                stop_words='english')
tf_vectorizer2 = CountVectorizer(max_df=0.95, min_df=2, max_features=100,
                                stop_words='english')

In [9]:
tf1 = tf_vectorizer1.fit_transform(np.array(Description_cleaned.replace(np.nan, '')))
tf2 = tf_vectorizer2.fit_transform(np.array(Search_Term.replace(np.nan,'')))
TF = TfidfTransformer()

In [10]:
#tf_idf_Bullets = TF.fit_transform(tf1)
tf_idf_Description=TF.fit_transform(tf1)
tf_idf_Search_Term= TF.fit_transform(tf2)

In [11]:
#Trans_Bullets=pd.DataFrame(tf_idf_Bullets.toarray())
#Trans_Bullets.index=Bullets.index
Trans_Description=pd.DataFrame(tf_idf_Description.toarray())
Trans_Description.index=description.index

In [12]:
Trans_Search_Term=pd.DataFrame(tf_idf_Search_Term.toarray())

In [13]:
#Trans_Bullets.head()
Trans_Description.head()

0   1   2   3   4   5   6         7   8         9   10  11  \
product_uid                                                                     
100001       0.000000   0   0   0   0   0   0  0.000000   0  0.000000   0   0   
100002       0.000000   0   0   0   0   0   0  0.140434   0  0.000000   0   0   
100003       0.200641   0   0   0   0   0   0  0.000000   0  0.000000   0   0   
100004       0.000000   0   0   0   0   0   0  0.000000   0  0.000000   0   0   
100005       0.000000   0   0   0   0   0   0  0.000000   0  0.190483   0   0   

             12        13  14  15  16  17  18        19      
product_uid                                                  
100001        0  0.000000   0   0   0   0   0  0.000000 ...  
100002        0  0.176037   0   0   0   0   0  0.123788 ...  
100003        0  0.000000   0   0   0   0   0  0.000000 ...  
100004        0  0.000000   0   0   0   0   0  0.000000 ...  
100005        0  0.000000   0   0   0   0   0  0.141314 ...  

[5 rows x 200 columns]

In [14]:
Trans_Search_Term.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...


In [15]:
import xgboost as xgb

In [16]:
train.index=train['product_uid']
Trans_Search_Term=Trans_Search_Term[0:len(train)]
Trans_Search_Term.index=train['product_uid']
Trans_Search_Term['relevance']=train['relevance']

In [17]:
#train_vec=pd.merge(Trans_Bullets,Trans_Search_Term,left_index=True,right_index=True)
train_vec=pd.merge(Trans_Description,Trans_Search_Term,left_index=True,right_index=True)

In [18]:
Relevance=train_vec['relevance']
train_vec=train_vec.drop(['relevance'],axis=1)

In [49]:
param={}
param['eta']=0.01
param['max_depth']=6
param['silent']=1
param['eval_metric']='rmse'
param['min_child_weight']=3
param['subsample']=0.7
param['colsample_bytree']=0.7
num_rounds=50000

In [37]:
train_vec=train_vec.reset_index().drop('product_uid',axis=1)

Relevance=Relevance.reset_index().drop('product_uid',axis=1)

In [60]:
start_ = time.time()

#x_train = np.array(train_vec.iloc[0:50000])
#y_train = np.array(Relevance.iloc[0:50000])

#x_validation = np.array(train_vec.iloc[50000:])
#y_validation = np.array(Relevance.iloc[50000:])
x_train, x_validation, y_train, y_validation=model_selection.train_test_split(train_vec,Relevance,test_size=0.3)

xgtrain = xgb.DMatrix(x_train, label= y_train)
xgvalidation=xgb.DMatrix(x_validation,label=y_validation)

In [63]:
clf = xgb.train(param, xgtrain, num_rounds,evals=[ (xgtrain,'train'),(xgvalidation,'eval')],
                early_stopping_rounds=100, verbose_eval =100)

[0]	eval-rmse:1.94156	train-rmse:1.93625
[100]	eval-rmse:0.865859	train-rmse:0.863035
[200]	eval-rmse:0.583134	train-rmse:0.581838
[300]	eval-rmse:0.531763	train-rmse:0.530081
[400]	eval-rmse:0.523276	train-rmse:0.520355
[500]	eval-rmse:0.52133	train-rmse:0.516668
[600]	eval-rmse:0.520428	train-rmse:0.514026
[700]	eval-rmse:0.519736	train-rmse:0.511615
[800]	eval-rmse:0.519148	train-rmse:0.509473
[900]	eval-rmse:0.518678	train-rmse:0.507393
[1000]	eval-rmse:0.518243	train-rmse:0.505427
[1100]	eval-rmse:0.517871	train-rmse:0.503659
[1200]	eval-rmse:0.517487	train-rmse:0.501803
[1300]	eval-rmse:0.517177	train-rmse:0.500047
[1400]	eval-rmse:0.516874	train-rmse:0.498383
[1500]	eval-rmse:0.516589	train-rmse:0.496676
[1600]	eval-rmse:0.516348	train-rmse:0.495065
[1700]	eval-rmse:0.516106	train-rmse:0.493462
[1800]	eval-rmse:0.515908	train-rmse:0.491982
[1900]	eval-rmse:0.515686	train-rmse:0.490421
[2000]	eval-rmse:0.515501	train-rmse:0.488972
[2100]	eval-rmse:0.515294	train-rmse:0.487504
[22

In [64]:
xgvalidation = xgb.DMatrix(x_validation)
y_prob = clf.predict(xgvalidation)

print 'Time elapsed: %.2f seconds' % (time.time() - start_)

Time elapsed: 2805.84 seconds
